# Library

In [ ]:
import ast
import json
import os
from glob import glob
from typing import List

import numpy as np
import pandas as pd
from tqdm import tqdm

tqdm.pandas()

# Config

In [ ]:
# Donwload input datasets from https://fielddaylab.wisc.edu/opengamedata/
INPUT_DIR = "/home/jovyan/work/input/raw/event_log"
OUTPUT_DIR = "/home/jovyan/work/input/preprocess/from_event_log_sample"

# Processing Function

## Constant

In [ ]:
EVENT_DATA_COLUMNS = [
    'event_custom',
    'fullscreen',
    'cur_cmd_fqid',
    'type',
    'save_code',
    'question',
    'response_index',
    'music',
    'question_index',
    'server_time',
    'hq',
    'response',
    'fqid',
    'subtype',
    'script_version',
    'script_type',
    'page',
    'cur_cmd_type',
    'remote_addr',
    'persistent_session_id',
    'http_user_agent',
    'screen_coor',
    'room_fqid',
    'name',
    'room_coor',
    'end_time',
    'interacted_fqid',
    'start_time',
    'text_fqid',
    'level',
    'text',
    'quiz_number'
]

NG_FQIDS = [
    ['tunic.capitol_0.hall.boss.chap1_finale_0_fail'],
    ['tunic.capitol_0.hall.boss.chap1_finale_1_fail'],
    ['tunic.capitol_0.hall.boss.chap1_finale_plaquefirst_0_fail',  'tunic.capitol_0.hall.boss.chap1_finale_slipfirst_0_fail'],
    ['tunic.capitol_1.hall.boss.chap2_finale_0_fail'],
    ['tunic.capitol_1.hall.boss.chap2_finale_1_fail'],
    ['tunic.capitol_1.hall.boss.chap2_finale_2_fail'],
    ['tunic.capitol_1.hall.boss.chap2_finale_3_fail'],
    ['tunic.capitol_1.hall.boss.chap2_finale_4_fail'],
    ['tunic.capitol_1.hall.boss.chap2_finale_5_fail'],
    ['tunic.capitol_1.hall.gramps.chap2_teddy_finale_0_fail'],
    ['tunic.capitol_1.hall.boss.chap2_teddy_finale_1_fail'],
    ['tunic.capitol_1.hall.gramps.chap2_teddy_finale_2_fail'],
    ['tunic.capitol_1.hall.wells.chap2_teddy_finale_3_fail'],
    ['tunic.capitol_2.hall.boss.chap4_finale_0_fail'],
    ['tunic.capitol_2.hall.boss.chap4_finale_1_fail'],
    ['tunic.capitol_2.hall.boss.chap4_finale_2_fail'],
    ['tunic.capitol_2.hall.boss.chap4_finale_3_fail'],
    ['tunic.capitol_2.hall.boss.chap4_finale_4_fail']
]

OK_FQIDS = [
    ['tunic.capitol_0.hall.boss.chap1_finale_1'],
    ['tunic.capitol_0.hall.boss.chap1_finale_plaquefirst_0', 'tunic.capitol_0.hall.boss.chap1_finale_slipfirst_0'],
    ['tunic.capitol_0.hall.boss.chap1_finale_plaquefirst_1', 'tunic.capitol_0.hall.boss.chap1_finale_slipfirst_1'],
    ['tunic.capitol_1.hall.boss.chap2_finale_1'],
    ['tunic.capitol_1.hall.boss.chap2_finale_2'],
    ['tunic.capitol_1.hall.boss.chap2_finale_3'],
    ['tunic.capitol_1.hall.boss.chap2_finale_4'],
    ['tunic.capitol_1.hall.boss.chap2_finale_5'],
    ['tunic.capitol_1.hall.boss.chap2_finale_6'],
    ['tunic.capitol_1.hall.boss.chap2_teddy_finale_1'],
    ['tunic.capitol_1.hall.gramps.chap2_teddy_finale_2'],
    ['tunic.capitol_1.hall.wells.chap2_teddy_finale_3'],
    ['tunic.capitol_1.hall.gramps.chap2_teddy_finale_4'],
    ['tunic.capitol_2.hall.boss.chap4_finale_1'],
    ['tunic.capitol_2.hall.boss.chap4_finale_2'],
    ['tunic.capitol_2.hall.boss.chap4_finale_3'],
    ['tunic.capitol_2.hall.boss.chap4_finale_4'],
    ['tunic.capitol_2.hall.boss.chap4_finale_5']
]

FEATURE_COLUMNS = ['session_id', 'index', 'elapsed_time', 'event_name', 'name',
'level', 'page', 'room_coor_x', 'room_coor_y', 'screen_coor_x',
'screen_coor_y', 'hover_duration', 'text', 'fqid', 'room_fqid',
'text_fqid', 'fullscreen', 'hq', 'music', 'level_group']

REPLACE_WORDS_PAIRS =[
    ('ðŸ˜´', '\\u00f0\\u0178\\u02dc\\u00b4'),
    ('I used to have a magnifying glass around hereâ€¦',
    'I used to have a magnifying glass around here\\u00e2\\u20ac\\u00a6'),
    ('"Taxidermy: the art of preparing, stuffing, and mounting the skins of animals."',
    '\\Taxidermy: the art of preparing, stuffing, and mounting the skins of animals.\\'),
    ('Great Scott, youâ€™re right!', "Great Scott, you're right!"),
    ('Iâ€™ve got Wellsâ€™s ID!', "I've got Wells's ID!"),
    ('Itâ€™s locked!', "It's locked!"),
    ('ðŸ˜\xad', '\\u00f0\\u0178\\u02dc\\u00ad'),
    ('Hang on. Iâ€™ll get you out of there!',
    "Hang on. I'll get you out of there!"),
    ('â\x9d¤ï¸\x8f', '\\u00e2\\u009d\\u00a4\\u00ef\\u00b8\\u008f'),
    ('ðŸ˜\x90', '\\u00f0\\u0178\\u02dc\\u0090'),
    ('ðŸ˜\x9d', '\\u00f0\\u0178\\u02dc\\u009d'),
    ('ðŸ§˜', '\\u00f0\\u0178\\u00a7\\u02dc'),
    ('ðŸ\x8d©', '\\u00f0\\u0178\\u008d\\u00a9'),
    ('ðŸ¦—', '\\u00f0\\u0178\\u00a6\\u2014'),
    ('ðŸ˜Š', '\\u00f0\\u0178\\u02dc\\u0160'),
    ('Ooh... "Ecology flag, by Ron Cobb."',
    'Ooh... \\Ecology flag, by Ron Cobb.\\'),
    ('ðŸ™„', '\\u00f0\\u0178\\u2122\\u201e'),
    ('He\'s wrong about old shirts and his name rhymes with "smells"...',
    "He's wrong about old shirts and his name rhymes with \\smells\\..."),
    ("Heâ€™s always trying to get you in trouble, and he doesn't like animals!",
    "He's always trying to get you in trouble, and he doesn't like animals!"),
    ('"Ecology flag, by Ron Cobb."', '\\Ecology flag, by Ron Cobb.\\')
 ]

## Function

In [ ]:
def decode_raw_log_data(df_log: pd.DataFrame) -> pd.DataFrame:
    """Load raw event data and convert to structured data

    Args:
        df_log (pd.DataFrame): Raw log data.

    Returns:
        pd.DataFrame: Structured log data.
    """
    df_log = df_log.rename(columns={"client_time": "timestamp", "event_data_complex": "event_data"})
    
    # Convert event_data column values from str to dict.
    # Whether ast or json should be used depends on the data.
    use_ast = (df_log["event_data"].str.contains("\"fqid\"")).sum() / len(df_log) < 0.7
    if use_ast:
        df_log["event_data"] = [ast.literal_eval(d) for d in df_log["event_data"]]
    else:
        df_log["event_data"] = [json.loads(d) for d in df_log["event_data"]]
    
    # Extract log data pushed into one column into different columns for each item.
    for col in EVENT_DATA_COLUMNS:
        df_log[col] = df_log["event_data"].apply(lambda row: row.get(col, np.nan))
    return df_log


def create_label_dataframe(df_log: pd.DataFrame) -> pd.DataFrame:
    """Create label datframe.

    Args:
        df_log (pd.DataFrame): Structured log data.

    Returns:
        pd.DataFrame: Label dataframe.
    """
    df_correct = df_log[["session_id"]].drop_duplicates().set_index("session_id")
    df_uncorrect = df_correct.copy()

    # Judges correct and incorrect answers for each session_id x question.
    def count_correct(group, ok_fqids_target_q: List[str], ng_fqids_target_q: List[str]):
        return pd.Series([int((group["cur_cmd_fqid"].isin(ok_fqids_target_q)).sum() > 0), int((group["cur_cmd_fqid"].isin(ng_fqids_target_q)).sum() > 0)])

    for q in range(18):
        res = df_log.groupby("session_id").apply(lambda x: count_correct(x, OK_FQIDS[q], NG_FQIDS[q]))
        res = res.rename(columns={0: f"correct_{q+1}", 1: f"uncorrect_{q+1}"})
        df_correct = df_correct.join(res[f"correct_{q+1}"], how="left")
        df_uncorrect = df_uncorrect.join(res[f"uncorrect_{q+1}"], how="left")
    
    # Put np.nan, not 0 or 1, in the label for players who retired in the middle of the game.
    # correct: correct_{q} == 1 and uncorrect_{q} == 0
    # uncorrect: correct_{q} == 1 and uncorrect_{q} == 1
    # retire: correct_{q} == 0
    corrects = df_correct.values - df_uncorrect.values
    corrects = np.where(df_correct.values == 0, np.nan, corrects)

    # Convert to vertical holding.
    df_correct[df_correct.columns] = corrects
    df_correct = df_correct.reset_index()
    df_correct_vertical = pd.DataFrame(columns=["session_id", "correct"])

    for q in range(1, 19):
        df_correct_target_q = df_correct[["session_id", f"correct_{q}"]].rename(columns={f"correct_{q}": "correct"}).copy()
        df_correct_target_q["session_id"] = df_correct_target_q["session_id"].astype(str) + "_q" + str(q)
        df_correct_vertical = pd.concat([df_correct_vertical, df_correct_target_q], axis=0)
    df_correct_vertical = df_correct_vertical.sort_values("session_id")

    return df_correct_vertical


def clean_feature_columns(df_log: pd.DataFrame) -> pd.DataFrame:
    """Perform a simple cleansing of the dataframe.

    Args:
        df_log (pd.DataFrame): Structured log data.

    Returns:
        pd.DataFrame: Label dataframe.
    """
    # Preprocess coor columns.
    df_log[["room_coor_x", "room_coor_y"]] = df_log["room_coor"].progress_apply(pd.Series).set_axis(['room_coor_x', 'room_coor_y'], axis=1)
    df_log[["screen_coor_x", "screen_coor_y"]] = df_log["screen_coor"].progress_apply(pd.Series).set_axis(['screen_coor_x', 'screen_coor_y'], axis=1)

    # Typing.
    df_log["level"] = df_log["level"].astype(int)
    df_log["index"] = df_log["index"].astype(int)
    df_log["elapsed_time"] = df_log["elapsed_time"].astype(int)
    
    # Create level_group.
    df_log["level_group"] = np.nan
    df_log.loc[df_log["level"]<=4, "level_group"] = "0-4"
    df_log.loc[(df_log["level"]>4)&(df_log["level"]<=12), "level_group"] = "5-12"
    df_log.loc[(df_log["level"]>12)&(df_log["level"]<=22), "level_group"] = "13-22"

    # Other preprocess.
    df_log["hover_duration"] = df_log["end_time"] - df_log["start_time"]
    df_log["event_name"] = df_log["subtype"] + "_" + df_log["type"]
    df_log.loc[df_log["type"]=="checkpoint", "event_name"] = df_log["type"]
    df_log.loc[df_log["text"]=='null', "text"] = np.nan

    return df_log


def create_feature_dataframe(df_log: pd.DataFrame) -> pd.DataFrame:
    """Create feature datframe.

    Args:
        df_log (pd.DataFrame): Structured log data.

    Returns:
        pd.DataFrame: Label dataframe.
    """
    df_log["timestamp"] = pd.to_datetime(df_log["timestamp"])

    def groupby_func(df_target):
        # Drop some records.
        df_target = df_target[~df_target["type"].isin(["quizquestion", "endgame", "quiz"])]
        df_target = df_target[~df_target["event_custom"].isin([1, 11, 12, 23, 24])]
        df_target = df_target[df_target["name"]!="choice"]
        df_target = df_target[df_target["subtype"]!="wildcard"]
        df_target = df_target[df_target["room_fqid"]!="tunic.nelson.trail"]
        df_target = df_target[~df_target["level"].isin([23, 24])]

        df_target = df_target[~((df_target["subtype"]=="notebook")&(df_target["room_fqid"]=="tunic.capitol_0.hall"))]
        df_target = df_target[~((df_target["subtype"]=="notebook")&(df_target["room_fqid"]=="tunic.capitol_1.hall"))]
        df_target = df_target[~((df_target["subtype"]=="notebook")&(df_target["room_fqid"]=="tunic.capitol_2.hall"))]

        # Reindex.
        df_target["index"] = range(len(df_target))

        # Drop chap2_finale except checkpoint.
        df_target = df_target[(df_target["type"]=="checkpoint")|(df_target["fqid"]!="chap2_finale")]

        # Calculate elapsed_time.
        df_target["elapsed_time"] = (df_target["timestamp"]-df_target["timestamp"].min()).map(lambda x: 1000*x.total_seconds())
        df_target["elapsed_time"] = df_target["elapsed_time"].astype(int)

        # In each level group, drop after the checkpoint.
        # chap1
        df_1 = df_target[(df_target["type"]=="checkpoint")&(df_target["fqid"]=="chap1_finale_c")]
        if len(df_1)>0:
            end_time_1 = df_1["elapsed_time"].values[0]
            df_target = df_target[~((df_target["text_fqid"]=="tunic.capitol_0.hall.chap1_finale_c")&(df_target["elapsed_time"]>end_time_1))]

        # chap2
        df_2 = df_target[(df_target["type"]=="checkpoint")&(df_target["fqid"]=="chap2_finale_c")]
        if len(df_2)>0:
            end_time_2 = df_2["elapsed_time"].values[0]
            df_target = df_target[~((df_target["text_fqid"]=="tunic.capitol_1.hall.chap2_finale_c")&(df_target["elapsed_time"]>end_time_2))]

        # chap4
        df_4 = df_target[(df_target["type"]=="checkpoint")&(df_target["fqid"]=="chap4_finale_c")]
        if len(df_4)>0:
            end_time_4 = df_4["elapsed_time"].values[0]
            df_target = df_target[~((df_target["text_fqid"]=="tunic.capitol_2.hall.chap4_finale_c")&(df_target["elapsed_time"]>end_time_4))]

        # Take the offset of index.
        df_start = df_target[df_target["text_fqid"]=="tunic.historicalsociety.closet.intro"]["index"]
        if len(df_start) > 0:
            start_index = df_start.values[0]
            df_target["index"] = (df_target["index"]-start_index)
        else:
            df_target = pd.DataFrame()
        return df_target

    # Drop and cleanse unnecessary records.
    df_all = df_log.groupby("session_id").progress_apply(groupby_func)
    df_all = df_all.drop(columns=["session_id"]).reset_index().drop(columns="level_1")
    df_all = clean_feature_columns(df_all)
    df_all = df_all[FEATURE_COLUMNS]

    # Replace some texts.
    for i in range(len(REPLACE_WORDS_PAIRS)):
        before_word = REPLACE_WORDS_PAIRS[i][0]
        after_word = REPLACE_WORDS_PAIRS[i][1]
        df_all.loc[df_all["text"]==before_word, "text"] = after_word
    df_all.loc[df_all["fqid"]=="0", "fqid"] = np.nan
    return df_all


def create_feature_and_label_csv(input_file: str) -> None:
    """Create feature csv and label csv files from raw log data in tsv files.

    Args:
        input_file (str): tsv file path.
    """
    feature_fn = OUTPUT_DIR + "/" + input_file.split("/")[-2] + "_feature.csv"
    label_fn = OUTPUT_DIR + "/" + input_file.split("/")[-2] + "_label.csv"

    if os.path.isfile(feature_fn):
        print("Skip existed datasets:", input_file)
        return

    df_log = pd.read_table(input_file)

    if len(df_log) < 5:
        print("Skip few logs:", input_file)
        return

    df_log = decode_raw_log_data(df_log)
    df_label = create_label_dataframe(df_log)
    df_feature = create_feature_dataframe(df_log)

    df_label.to_csv(label_fn, index=False)
    df_feature.to_csv(feature_fn, index=False)

    print("Processed:", input_file)


# Processing

## Generate datasets from raw log data.

In [ ]:
os.makedirs(OUTPUT_DIR, exist_ok=True)

all_tsv = glob(f"{INPUT_DIR}/*/*/*.tsv")

# Sort by file size.
file_sizes = {}
for file_path in all_tsv:
    file_sizes[file_path] = os.path.getsize(file_path)
all_tsv = sorted(all_tsv, key=lambda x: file_sizes[x])

# Without multiprocessing ver because of memory limit.
for input_file in all_tsv:
    create_feature_and_label_csv(input_file)

## Concat all datasets into one

In [ ]:
features = glob(f"{OUTPUT_DIR}/*_feature.csv")
labels = glob(f"{OUTPUT_DIR}/*_label.csv")

df_features = pd.DataFrame()
for feature in tqdm(features):
    df_feature = pd.read_csv(feature)
    df_features = pd.concat([df_features, df_feature], axis=0)

df_labels = pd.DataFrame()
for label in tqdm(labels):
    df_label = pd.read_csv(label)
    df_labels = pd.concat([df_labels, df_label], axis=0)

df_features.to_csv(f"{OUTPUT_DIR}/all_features.csv", index=False)
df_labels.to_csv(f"{OUTPUT_DIR}/all_labels.csv", index=False)